# Spesifikasi

- V<sub>in</sub> = 48V
- V<sub>out</sub> = 12V
- P<sub>o</sub> = 100W
- &#916;V<sub>o</sub> &#8804; 1%
- &#916;I<sub>L</sub> &#8804; 10%
- f<sub>sw</sub> = 20e3
- Mosfet: IRFB4310PbF, Infineon

# Plecs

In [19]:
import xmlrpc.client as xml
import os
import matplotlib.pyplot as plt
import numpy as np
import itertools
import csv

In [3]:
model = 'buck'
file_type = '.plecs'
V_in = 48
V_out = 12
f_sw = 20e3

In [4]:
plecs = xml.Server("http://localhost:1080/RPC2").plecs

In [5]:
plecs.load(r"E:\OneDrive - UGM 365\Mata Kuliah\Skripsi\Plecs\Python\buck\buck.plecs")

0

In [6]:
plecs.get(model+'/L1')

[['Name', 'L1'],
 ['Type', 'Inductor'],
 ['CommentStatus', 'Active'],
 ['L', '0.0007'],
 ['i_init', '0']]

In [7]:
plecs.get(model+'/C')

[['Name', 'C'],
 ['Type', 'Capacitor'],
 ['CommentStatus', 'Active'],
 ['C', '6e-05'],
 ['v_init', '0']]

In [8]:
plecs.get(model+'/Symmetrical PWM1')

[['Name', 'Symmetrical PWM1'],
 ['Type', 'Subsystem'],
 ['CommentStatus', 'Active'],
 ['sampling', 'Natural (carrier starts at center)'],
 ['fc', '30000.0'],
 ['carrier_phaseshift', '0'],
 ['carrier_limits', '[0 1]'],
 ['output_values', '[-1 1]']]

In [9]:
plecs.get(model+'/RL')

[['Name', 'RL'],
 ['Type', 'Resistor'],
 ['CommentStatus', 'Active'],
 ['R', '1e-3']]

In [10]:
RL = plecs.get(model+'/RL', 'R')

In [11]:
plecs.get(model+'/ESR')

[['Name', 'ESR'],
 ['Type', 'Resistor'],
 ['CommentStatus', 'Active'],
 ['R', '0.025681987128706025']]

In [12]:
plecs.get(model+'/ESL')

[['Name', 'ESL'],
 ['Type', 'Inductor'],
 ['CommentStatus', 'Active'],
 ['L', '5.839164571365709e-08'],
 ['i_init', '0']]

In [22]:
L_range = (30e-6, 2000e-6)  # Range for L
C_range = (30e-6, 200e-6)   # Range for C
fsw_range = (20e3, 200e3)    # Range for fsw
num_values = 2           # Number of values for each parameter

# Borders for R_L
border_values = [
    (144e-6, 0.079576),
    (509e-6, 0.179228),
    (1003e-6, 0.253239),
    (2519e-6, 0.443394)
]

# Constants for ESL calculation
K_esl = 4.389e-11  # Adjust as needed
tan_delta = 0.14

# Create arrays of discrete L, C, and fsw values within the specified ranges
L_values = np.linspace(L_range[0], L_range[1], num=num_values)
C_values = np.linspace(C_range[0], C_range[1], num=num_values)
fsw_values = np.linspace(fsw_range[0], fsw_range[1], num=num_values)

# Initialize arrays to store results
average_currents = []
delta_currents = []
delta_currents_percentage = []

# Open a CSV file for writing
csv_file_path = 'simulation_results.csv'

# Define the header for the CSV file
csv_header = ['No','L', 'C', 'fsw', 'RL', 'ESR', 'ESL', 'delta_current_percentage']

# Initialize a list to store the data for each simulation
csv_data = []

# Print the chosen values
print("Chosen L values:", L_values)
print("Chosen C values:", C_values)
print("Chosen fsw values:", fsw_values)

# Generate all combinations of L, C, and fsw
combinations = list(itertools.product(L_values, C_values, fsw_values))

# Display the chosen values and simulate
print("\nChosen values and simulation results:")

Chosen L values: [3.e-05 2.e-03]
Chosen C values: [3.e-05 2.e-04]
Chosen fsw values: [ 20000. 200000.]

Chosen values and simulation results:


# Functions

## Write to CSV

In [16]:
def write_to_csv(file_path, header, data):
    with open(file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(header)
        csv_writer.writerows(data)

## Ripples

### Current
- Average
- &Delta;I<sub>L</sub>%

In [17]:
def calculate_current(times, current, start_time, end_time):
    # Convert 'times' to a NumPy array
    times = np.array(times)

    # Find the indices corresponding to the time range
    start_index = np.argmax(times >= start_time)
    end_index = np.argmax(times >= end_time)

    # Extract the current values in the specified time range
    current_range = current[start_index:end_index]

    # Calculate the average current value
    average_current = np.mean(current_range)

    # Calculate the difference between the highest and lowest current values
    delta_current = np.max(current_range) - np.min(current_range)

    # print(np.max(current_range))
    # print(np.min(current_range))
    delta_current_percentage = delta_current/average_current 

    return average_current, delta_current, delta_current_percentage

### &Delta;V<sub>O</sub>%

In [ ]:
def calculate_voltage(times, voltage, start_time, end_time):
    # Convert 'times' to a NumPy array
    times = np.array(times)

    # Find the indices corresponding to the time range
    start_index = np.argmax(times >= start_time)
    end_index = np.argmax(times >= end_time)

    # Extract the voltage values in the specified time range
    voltage_range = voltage[start_index:end_index]

    # Calculate the average voltage value
    average_voltage = np.mean(voltage_range)

    # Calculate the difference between the highest and lowest voltage values
    delta_voltage = np.max(voltage_range) - np.min(voltage_range)

    print('Voltage:')
    print("\nmax:", np.max(voltage_range))
    print("\nmin:", np.min(voltage_range))
    print("\ndelta:",delta_voltage)
    print("\naverage:",average_voltage)

    delta_voltage_percentage = delta_voltage/average_voltage * 100

    return average_voltage, delta_voltage, delta_voltage_percentage


# Loop

In [23]:
# Loop through the combinations and simulate
csv_data = []
for simulation_num, (L, C, fsw) in enumerate(combinations, start=1):
    # Rules for picking R_L based on L
    R_L = next((rl for border, rl in border_values if L <= border), 0.0)

    # Calculate ESR and ESL based on the updated C
    ESR = tan_delta / (2 * np.pi * fsw * C)
    ESL = K_esl / C

    # Set Plecs parameters and simulate
    plecs.set(model+'/L1', 'L', str(L))
    plecs.set(model+'/C', 'C', str(C))
    plecs.set(model+'/Symmetrical PWM1', 'fc', str(fsw))
    plecs.set(model+'/RL', 'R', str(R_L))
    plecs.set(model+'/ESR', 'R', str(ESR))
    plecs.set(model+'/ESL', 'L', str(ESL))

    times = plecs.simulate(model)['Time']
    current = plecs.simulate(model)['Values'][0]
    voltage = plecs.simulate(model)['Values'][1]

    # Calculate average and delta current using the function
    average_current, delta_current, delta_current_percentage = calculate_current(times, current, start_time=0.004, end_time=0.005)
    average_voltage, delta_voltage, delta_voltage_percentage = calculate_voltage(times, voltage, start_time=0.004, end_time=0.005)

    # Append the data for this simulation to the list
    csv_data.append([simulation_num,L, C, fsw, R_L, ESR, ESL, average_current, delta_current_percentage, delta_voltage_percentage])

    print(f'\nSimulation {simulation_num} for L={L:.6f}, C={C:.6f}, fsw={fsw:.2f}, RL={R_L:.6f}, ESR={ESR:.6f}, ESL={ESL:.6e}')
    print(f"\nSimulation {simulation_num} results:")
    print(f"Average Current between 0.004 and 0.005 seconds: {average_current} A")
    print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current} A")
    print(f"Difference between highest and lowest current values between 0.004 and 0.005 seconds: {delta_current_percentage} %")
    print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage} A")
    print(f"Difference between highest and lowest voltage values between 0.004 and 0.005 seconds: {delta_voltage_percentage} %")


Simulation 1 for L=0.000030, C=0.000030, fsw=20000.00, RL=0.079576, ESR=0.037136, ESL=1.463000e-06

Simulation 1 results:
Average Current between 0.004 and 0.005 seconds: 3.2556026550071735 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds: 14.944351228442894 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds (%): 4.590348642657058 %

Simulation 2 for L=0.000030, C=0.000030, fsw=200000.00, RL=0.079576, ESR=0.003714, ESL=1.463000e-06

Simulation 2 results:
Average Current between 0.004 and 0.005 seconds: 2.4480963015005166 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds: 1.4511352153038453 A
Difference between highest and lowest current values between 0.004 and 0.005 seconds (%): 0.5927606746574463 %

Simulation 3 for L=0.000030, C=0.000200, fsw=20000.00, RL=0.079576, ESR=0.005570, ESL=2.194500e-07

Simulation 3 results:
Average Current between 0.004 and 0.005 seconds: 2.2592

In [24]:
# Use the function to write data to the CSV file
write_to_csv(csv_file_path, csv_header, csv_data)